In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('data/ckd_clean.csv')
df.head()

,age,blood_pressure,specific_gravity,albumin,sugar,abnormal_red_blood_cells,abnormal_pus_cell,pus_cell_clumps,bacteria,blood_glucose_random,...,packed_cell_volume,white_blood_cell_count,red_blood_cell_count,hypertension,diabetes_mellitus,coronary_artery_disease,poor_appetite,peda_edema,aanemia,class
0,48.0,80.0,1.020,1.0,0.0,0.0,0.0,0.0,0.0,121.000000,...,44.0,7800.0,5.200000,1.0,1.0,0.0,0.0,0.0,0.0,1
1,7.0,50.0,1.020,4.0,0.0,0.0,0.0,0.0,0.0,103.333333,...,38.0,6000.0,5.733333,0.0,0.0,0.0,0.0,0.0,0.0,1
2,62.0,80.0,1.010,2.0,3.0,0.0,0.0,0.0,0.0,423.000000,...,31.0,7500.0,2.833333,0.0,1.0,0.0,1.0,0.0,1.0,1
3,48.0,70.0,1.005,4.0,0.0,0.0,1.0,1.0,0.0,117.000000,...,32.0,6700.0,3.900000,1.0,0.0,0.0,1.0,1.0,1.0,1
4,51.0,80.0,1.010,2.0,0.0,0.0,0.0,0.0,0.0,106.000000,...,35.0,7300.0,4.600000,0.0,0.0,0.0,0.0,0.0,0.0,1


In [4]:
# Activation functions and derivatives
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

# Loss function (Binary Cross-Entropy)
def binary_cross_entropy(y_true, y_pred):
    epsilon = 1e-8  # Avoid log(0)
    return -np.mean(y_true * np.log(y_pred + epsilon) + (1 - y_true) * np.log(1 - y_pred + epsilon))

def binary_cross_entropy_derivative(y_true, y_pred):
    epsilon = 1e-8
    return (y_pred - y_true) / ((y_pred * (1 - y_pred)) + epsilon)

# MLP class
class MLPBinaryClassifier:
    def __init__(self, input_size, hidden_layers, activation="relu", learning_rate=0.01, epochs=1000):
        self.input_size = input_size
        self.hidden_layers = hidden_layers
        self.activation_func = relu if activation == "relu" else sigmoid
        self.activation_derivative = relu_derivative if activation == "relu" else sigmoid_derivative
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = []
        self.biases = []
        self._initialize_weights()

    def _initialize_weights(self):
        layer_sizes = [self.input_size] + self.hidden_layers + [1]
        for i in range(len(layer_sizes) - 1):
            self.weights.append(np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * 0.01)
            self.biases.append(np.zeros((1, layer_sizes[i + 1])))

    def forward(self, X):
        activations = [X]
        for i in range(len(self.weights) - 1):
            X = self.activation_func(np.dot(X, self.weights[i]) + self.biases[i])
            activations.append(X)
        output = sigmoid(np.dot(X, self.weights[-1]) + self.biases[-1])
        activations.append(output)
        return activations

    def backward(self, activations, y):
        grads_w = []
        grads_b = []
        
        # Compute loss gradient
        delta = binary_cross_entropy_derivative(y, activations[-1]) * sigmoid_derivative(activations[-1])
        
        for i in reversed(range(len(self.weights))):
            grads_w.insert(0, np.dot(activations[i].T, delta))
            grads_b.insert(0, np.sum(delta, axis=0, keepdims=True))
            if i > 0:
                delta = np.dot(delta, self.weights[i].T) * self.activation_derivative(activations[i])
        
        return grads_w, grads_b
    
    def update_weights(self, grads_w, grads_b):
        for i in range(len(self.weights)):
            self.weights[i] -= self.learning_rate * grads_w[i]
            self.biases[i] -= self.learning_rate * grads_b[i]
    
    def train(self, X, y):
        for epoch in range(self.epochs):
            activations = self.forward(X)
            grads_w, grads_b = self.backward(activations, y)
            self.update_weights(grads_w, grads_b)
            if epoch % 100 == 0:
                loss = binary_cross_entropy(y, activations[-1])
                print(f"Epoch {epoch}, Loss: {loss:.4f}")
    
    def predict(self, X):
        return (self.forward(X)[-1] > 0.5).astype(int)

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df.drop(columns=["class"]).values
y = df["class"].values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

/Users/jeffwang/miniforge3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


((320, 24), (80, 24), (320, 1), (80, 1))

In [6]:
mlp = MLPBinaryClassifier(input_size=24, hidden_layers=[16, 8], activation="relu", learning_rate=0.01, epochs=1000)

mlp.train(X_train, y_train)

predictions = mlp.predict(X_test)
accuracy = np.mean(predictions == y_test)
print(f"Test Accuracy: {accuracy:.4f}")

Epoch 0, Loss: 0.6931
Epoch 100, Loss: 0.0069
Epoch 200, Loss: 0.0023
Epoch 300, Loss: 0.0014
Epoch 400, Loss: 0.0010
Epoch 500, Loss: 0.0007
Epoch 600, Loss: 0.0006
Epoch 700, Loss: 0.0005
Epoch 800, Loss: 0.0004
Epoch 900, Loss: 0.0004
Test Accuracy: 0.9875
